In [3]:
from FR3Py.sim.mujoco import FR3Sim

In [4]:
robot = FR3Sim(interface_type='velocity')

In [5]:
import numpy as np
import time
for i in range(100000):
    robot.step()
    robot.setCommands(np.array([0, 0, 0, 0, 0, 0, 0]))
    time.sleep(robot.dt)

In [1]:
robot.getJointStates()

NameError: name 'robot' is not defined

In [2]:
import time
import numpy as np
from FR3Py.controllers.jacobianPseudoInv import WaypointController

controller = WaypointController(kp=1.5)
# Read the initial state of the robot
time.sleep(0.2)
robot.reset()
state = robot.getJointStates()
if state is not None:
    print('starting the controller')
    q, dq = state['q'], state['dq']
    q = np.hstack([q, np.zeros(2)])
    dq = np.hstack([dq, np.zeros(2)]
                   )
    p0 = controller.robot.getInfo(q,dq)['P_EE']
    R0 = controller.robot.getInfo(q,dq)['R_EE']
    T0 = np.vstack([np.hstack([R0, p0.reshape(3,1)]), np.array([0,0,0,1])])
    
    start_time = time.time()
    while time.time()-start_time < 10:
        time.sleep(robot.dt)
        t = time.time() - start_time
        x = 0.1 * np.sin(t)
        T= T0 @ np.vstack([np.hstack([np.eye(3), np.array([0,0,x]).reshape(3,1)]), np.array([0,0,0,1])])
        state = robot.getJointStates()
        q, dq = state['q'], state['dq']
        cmd = controller.compute(q, dq, T_cmd=T)
        robot.setCommands(cmd[:7])
        robot.step()

    robot.setCommands(np.zeros(7))
    print('Demo ended.')
else:
    print('State is None. check the connection')

NameError: name 'robot' is not defined